### 第三章 Processing 交互基础

#### 本章内容

1. 交互系统核心框架
2. 事件驱动的交互系统
3. 使用Processing实现鼠标与键盘交互

#### 1. 交互核心框架

现在你已经能够用Processing（或者Py5）绘制一些简单的图形了，相信你对“让计算机画出圆、方块、线条”已经不陌生。不过，你可能已经注意到，之前的代码里总是**不停、重复**出现过两个特别的方法：`setup()` 和 `draw()`。这两个函数，其实就是整个Processing（或者Py5）实现交互和动画的“双核心发动机”——没有它们，画布就只能安静地待在那里，啥也不动。

那么，假设我们需要自己设计一个可以捕捉鼠标路径的交互界面，这就得科学地用好这两个工具。我们先来仔细拆解一下整个过程。首先，任何一个动态交互程序，都需要对整个系统做一次初始化，比如设置画布的大小（想象舞台有多大）、确定背景颜色、建立一些初始数据（比如存放鼠标轨迹的内存空间）等等。这些准备工作最关键的一点是——**它们只需要干一次**，有点像剧场开演前的布景：布景师把道具都摆好，灯光调对，观众才有得看。在Py5里，这样的事情，都放进 `setup()` 函数里去做。

舞台搭好之后，真正的“戏”可就开始了。用户拿起鼠标，在屏幕上晃晃悠悠地滑动，我们的程序要做的，就是**不停地**、一遍遍地获取鼠标的当前位置，每次都在那个新位置画一个“小圆点”，这样小圆连成线，轨迹就出来了。注意：这种操作的特点，就是**不断重复、一直发生**，而不是静态地只干一次。想一想，你每打一次游戏，每画一次动画，都是同样的逻辑。这个“重复性任务”的好帮手，就是 `draw()`。你可以把它想成一个超级勤快的摄影师，每隔一小段时间“咔嚓”一下，把舞台当前的画面拍下来，然后立刻准备好下一帧。

那么，**draw()执行的速度是固定的吗？能不能快点或者慢点？** 这就涉及到另一个被称为**刷新率**的概念，英文名叫 Frame Rate，单位通常是每秒多少帧（FPS，frames per second）。

刷新率其实就像一部电影的“播放速度”。如果你每秒放24张图片（即24帧），动画看起来就很流畅。这24，就是刷新率。如果我们把刷新率调得很慢，比如1帧/秒，你每一秒只看到一张图片，动画效果就像幻灯片：一卡一卡地闪。如果把刷新率调得飞快，比如60帧/秒，你几乎捕捉不到每一帧的切换，看起来就像现实世界一样平滑。所以，**刷新率越高，动画越流畅；刷新率越低，动画就越容易出现闪烁和断续**。

在Py5里，我们可以通过 `py5.frame_rate` 这个函数，把刷新率设成30，也就是每秒调用 `draw()` 30次。你还可以根据交互场景自由地调整这个值，这样，既能让动画流畅，又不会让电脑CPU“累趴下”。

其实，这里跟你平时看到的动画片、电子游戏，甚至手机屏幕的显示原理都是一样的。动画的原理，就是把一帧帧静止的画面快速播放，让人产生连续运动的错觉。电子游戏里的“帧率”更是玩家津津乐道的话题；帧数高，操作手感就丝滑。甚至你玩音乐可视化、数据仪表盘、实时代码艺术时，`draw()`的刷新率都决定了“画面能不能实时响应”。

- 名词汇总：
  - setup：Processing应用程序的初始化函数，只在程序启动时调用一次
  - draw：Processing的主循环函数，程序系统后，根据frame_rate的设定，不停的被调用
  - 刷新率：最开始指屏幕显示图片的速度，但是为Hz或FPS，Processing中刷新率指draw函数每秒钟被调用的次数

In [ ]:
'''
这段代码请放置到单独的.py文件中运行
或者
需要重启Jupyter内核后再运行
因为前文的Setup函数已经执行过了，会覆盖这里面的frame_rate设置
'''

'''
import py5

pos = 0


def setup():
    py5.frame_rate(4) # 通过修改刷新率，使得线条绘制速度变慢，也就是draw()函数每秒只执行4次


def draw():
    global pos
    py5.background(204)
    pos += 1
    py5.line(pos, 20, pos, 80)
    if pos > py5.width:
        pos = 0

py5.run_sketch()  # 启动Py5程序，执行setup()和draw()函数
'''

#### 2. 事件驱动

##### 2.1 事件驱动程序设计



如果你曾经有过其他程序开发经验，比如用过MFC、.NET、JavaScript、Android或者Apple平台，你一定还记得一种非常常见的编程方式：**回调函数（Callback）**。所谓回调函数，就是我们先提前写好一段“处理某种情况的代码”，然后告诉系统：“喂！以后只要发生这种事，就帮我自动调用这段代码！”比如按钮被点击、鼠标被按下、键盘敲击等。这种模式在不同的开发平台上虽然具体语法有差异，但思想其实是一样的。如果你还没碰过这些经验也没关系，我们完全可以从头来理解这个原理。

现在，回到我们最开始的那个例子：如何追踪并绘制用户的鼠标轨迹。整个实现过程其实很简单，但背后正是一套典型的事件驱动机制。

当用户在程序界面上**按下鼠标**，首先需要有一个程序模块能够**捕获并记录用户当前位置**。这一步背后，系统其实自动帮我们监控了鼠标的状态，每当发现“鼠标被按下”这个特殊情况（即**事件**）发生时，就“触发”了一个事先和这个事件关联好的代码区块，用来记录鼠标位置，并把坐标信息保存到一个合适的数据结构中（比如队列或列表）。接下来，每次界面刷新的时候，程序就可以遍历所有已保存的坐标点，并在这些点上绘制圆点或线条，最终在窗口上还原鼠标的移动轨迹。整个过程的核心，其实就是“特定事件—>特定处理”，而不需要不停地盲目检查鼠标状态，这就是**事件驱动**的高效之处。

咱们再设计另一个交互应用场景——比如做一个“键盘映射显示”，要求无论用户按下键盘上哪个键，屏幕上就有一个对应位置的色块高亮显示。实现这个效果的典型步骤如下：
1. 首先要有一个方法，用来响应“键盘被按下”的情况，这个方法需要读取用户具体按下了哪个键。
2. 根据这个按键的编码或字符，将它映射到屏幕上的某个具体坐标（比如虚拟键盘的位置，或者一串色块的排列）。
3. 然后在屏幕上那个位置绘制一个色块或高亮标记，实时反馈给用户。

这两种常见的交互设计最大的共同点，就是**都要有一个专门处理用户操作的方法（即回调函数）**，以及某种“机制”能帮我们自动在用户每次操作时**调用这些方法**。这个被用户操作“触发”的动作（比如鼠标按下、键盘敲击等）在理论里就叫做**事件（Event）**；而你用来处理这些具体事件的专用方法，就叫做**事件回调函数（Event Callback）**。系统为了能够可靠、高效地处理大量的事件，通常还会维护一个**事件循环队列（Event Loop Queue）**，它把所有发生过的事件（无论是鼠标、键盘还是其他用户操作）一个个排队，等待依次处理。**事件循环队列**的最大作用，就是保证不管你同时按了多少键、点了多少下鼠标，每个动作都不会“丢失”或者“抢占”，而是按照顺序一个个被处理，让用户体验变得丝滑、直观。

这一整套结合了事件、回调函数和循环队列的技术体系，被称为**事件驱动编程（Event-driven Programming）**。它广泛存在于各种现代交互软件、操作系统、图形界面库甚至服务器和物联网设备中。其主要架构可以分为几个层次：首先是**事件源（Event Source）**，也就是各种能发出动作信号的实体（如鼠标、键盘、窗口等）；其次是**事件分发器（Event Dispatcher）**，负责捕捉和识别各种事件类型；再往下是**事件队列**，负责存储即将被处理的事件；然后是**事件循环（Event Loop）**，专门监测队列内事件是否到来，并负责依次分发；最终轮到**事件处理器（Event Handler）**，也就是我们预先注册好的各类回调函数，具体应对每种交互动作。

实际应用中，基本所有的计算机交互设计设备最常见的事件类型包括：鼠标按下、鼠标松开、鼠标移动、鼠标拖拽、鼠标滚轮、鼠标双击、键盘按下、键盘松开、触摸屏点击、窗口大小调整、窗口获得/失去焦点等。这些事件每发生一次，系统都会按照“事件源→分发→队列→循环→处理器”的链条进行一次传递。

以**鼠标按下（Mouse Down）**为例，详细的事件交互机制大致可分为：
- 操作系统实时监听到物理鼠标传感器的“点击”动作后，立即生成一个“鼠标按下”事件对象；
- 这个事件对象会被添加到应用程序的“事件队列”里；
- 稍后轮到应用程序的事件循环检测到它，就会检索和这个事件类型绑定的处理器（也就是回调函数），并把事件具体数据（如鼠标坐标、哪个按键）传递进去；
- 事件处理器运行完毕后，应用程序界面立刻反馈出相应的视觉或状态变化，如按钮高亮、绘图开始等。
  
整个流转过程高效且有序，充分体现了**事件驱动编程**的理论基础与工程实用价值。这也是现代所有交互式系统不可或缺的“底层灵魂”。



##### 2.2 Processing 鼠标键盘交互编程基础



Processing中，已经大幅简化了事件驱动编程的实现过程。它为我们提供了许多已经写好的**事件框架**和机制，让开发者不用深究底层事件队列或者手动注册复杂的监听器，而只需要**按照规定的名字写好对应的事件处理函数**，系统就会自动帮你把事件和代码连接起来。例如，只要你定义一个函数名为 `draw`，Processing就会自动在每一帧刷新时调用它，让你专心负责“要绘制什么”，而不用自己搭建事件循环。可以说，`draw`其实就是一个由系统自动调度的“定时事件回调”，每隔固定时间帮你重绘屏幕，非常适合实现动画和实时交互效果。

除此之外，像鼠标的坐标、鼠标按键状态、键盘按下情况等各种常见的输入信息，Processing和Py5也会**自动帮你实时保存起来**，并以简明直接的属性名暴露出来，比如 `mouse_x`、`mouse_y`、`pmouse_x`（上一次的位置）、`mouse_pressed`（鼠标是否按下）、`key`（最近按过的键）、`key_code`（具体的按键编号）等。你只需要**直接访问这些属性**，随时可以获得最新的用户交互数据，无需自己手动监听或者维护复杂的全局变量。

为了方便大家查阅和记忆，这里列出一些在Py5或Processing中常见的**鼠标与键盘基本事件函数**，以及它们的主要参数含义：

| 事件函数        | 触发时机                            | 参数说明（如有）                                           |
|----------------|-------------------------------------|----------------------------------------------------------|
| mouse_pressed  | 当鼠标任意按键被按下时触发          | event（可省略），包含button（左键/右键）、pressed（状态）等|
| mouse_released | 当鼠标任意按键被释放时触发           | event（同上）                                             |
| mouse_moved    | 当鼠标在窗口内移动（未按任何键）时触发| event（含x, y坐标等）                                      |
| mouse_dragged  | 当按下鼠标键并拖动时触发             | event（同上）                                             |
| key_pressed    | 当任意键盘按键被按下时触发           | event，包含key（字符）、key_code（键码）                   |
| key_released   | 当任意键盘按键被释放时触发           | event，包含key、key_code                                  |

> 注意：这些事件函数的名字是**预设的且区分大小写**，写错就不会被系统自动调用。大多数情况下，如果你的事件处理函数定义在全局作用域，Processing/Py5会自动帮你注册好，无需手动绑定。

- 名词汇总
  - **事件（Event）：**  
  程序运行时，由用户操作或系统变化等触发的特定“动作”或“信号”。

  - **事件源（Event Source）：**  
  能够产生事件的对象或设备，比如鼠标、键盘、按钮等。

  - **事件回调函数（Event Callback/Handler）：**  
  专门用来响应和处理某类事件的函数或方法，由系统自动调用。

  - **事件分发器（Event Dispatcher）：**  
  负责将捕获到的事件分派到相应回调函数的中介机制。

  - **事件队列（Event Queue）：**  
  用于按顺序存放等待被处理的所有事件的特殊数据结构。

  - **事件循环（Event Loop）：**  
  不断监视事件队列，并依次调度各类事件处理程序的核心机制。

  - **事件驱动编程（Event-driven Programming）：**  
  一种以事件和回调机制为主，程序随事件触发自动响应的编程模式。

In [ ]:
import py5

# 用于存储鼠标轨迹的列表，每一项是一个(x, y)坐标元组
mouse_path = []

# 标记当前是否在绘制状态
drawing = False

def setup():
    py5.size(600, 400)
    py5.background(255)  # 初始为白色背景
    py5.stroke_weight(3) # 轨迹线稍微粗一点
    my_font = py5.create_font("Noto Sans Thin", 12)
    py5.text_font(my_font)
    py5.text_align(py5.LEFT, py5.TOP)

def draw():
    py5.background(255)    # 每帧清理背景，避免拖影
    py5.stroke(20, 90, 200)
    # 遍历轨迹数组，逐段画线
    for i in range(1, len(mouse_path)):
        pt1 = mouse_path[i-1]
        pt2 = mouse_path[i]
        py5.line(pt1[0], pt1[1], pt2[0], pt2[1])

    # 用文字提醒如何操作
    py5.fill(0)
    py5.text_size(18)
    if not drawing:
        py5.text('按住鼠标左键在屏幕上拖动来绘制轨迹', 20, 30)

def mouse_pressed():
    global drawing
    if py5.mouse_button == py5.LEFT:
        drawing = True
        mouse_path.clear()      # 每次新按下时，清理旧轨迹
        mouse_path.append((py5.mouse_x, py5.mouse_y))  # 记录起点

def mouse_released():
    global drawing
    if py5.mouse_button == py5.LEFT:
        drawing = False

def mouse_dragged():
    # 只有drawing为True才记录点，避免非绘制时乱记录
    if drawing:
        mouse_path.append((py5.mouse_x, py5.mouse_y))

py5.run_sketch()

In [ ]:
import py5
import string    # 用于获得英文字母列表

# 定义要显示的按键集合（这里只显示A-Z）
key_rows = [
    list("QWERTYUIOP"),
    list("ASDFGHJKL"),
    list("ZXCVBNM")
]
cell_w, cell_h = 55, 55   # 每个格子的宽高
cell_margin = 15          # 格子间的间距

# 记录当前已按下的键（用大写字符来存储，便于匹配）
pressed_keys = set()

def setup():
    py5.size(800, 400)
    my_font = py5.create_font("Noto Sans Thin", 12)
    py5.text_font(my_font)
    py5.text_align(py5.CENTER, py5.CENTER)

def draw():
    py5.background(240)
    y = 50
    for row in key_rows:
        x = 30
        for k in row:
            # 如果正在按这个键，画高亮色，否则画灰色背景
            if k in pressed_keys:
                py5.fill(120, 220, 120)
            else:
                py5.fill(220)
            py5.stroke(80)
            py5.rect(x, y, cell_w, cell_h, 10)  # 带圆角的色块
            # 画出字母
            py5.fill(60)
            py5.text(k, x + cell_w/2, y + cell_h/2)
            x += cell_w + cell_margin
        y += cell_h + cell_margin
    # 底部说明
    py5.fill(60)
    py5.text_size(16)
    py5.text("请直接按下键盘上的英文字母（A-Z）", py5.width/2, py5.height-25)

def key_pressed():
    # 将用户按下的字母转为大写进行匹配，如果有效则添加到pressed_keys
    k = py5.key.upper()
    for row in key_rows:
        if k in row:
            pressed_keys.add(k)

def key_released():
    # 松开键也要同步移除色块
    k = py5.key.upper()
    pressed_keys.discard(k)

py5.run_sketch()

In [ ]:
import py5

red = True
mouseSpeed = 1

def setup():
    py5.size(500, 500)
    py5.stroke(255, 0, 0)

def draw():
    mouseSpeed = abs(py5.mouse_x - py5.pmouse_x)
    py5.stroke_weight(mouseSpeed)
    py5.line(py5.pmouse_x, py5.pmouse_y, py5.mouse_x, py5.mouse_y)

def mouse_pressed():
    py5.stroke(0 if red else 255, 255 if red else 0, 0)

py5.run_sketch()

#### 本章总结

##### 本章知识点汇总



1. 交互设计的基本概念
- **交互的定义**：  
交互设计是指设计人与计算机系统之间交流与操作的方式，使用户能够通过输入指令，从程序获得反馈，完成信息交换和功能控制。
- **交流的本质**：  
人与计算机的交流是“输入—处理—输出”的循环过程。用户发送操作信号（如点按、输入文本），系统识别后给出反馈（如界面变化、图像/声音反馈）。
- **常见交互场景**：  
如点击按钮、拖动物体、滑动屏幕、输入文本、手势操作、声音识别等。这些场景本质上都是捕捉用户某种动作，让程序做出对应反应。

2. 常见的交互技术与设备
- *鼠标*：可用于点击、拖动、滚轮操作，控制指针坐标。
- *键盘*：输入字符、命令、快捷键等。
- *触摸屏*：支持手指点击、滑动、缩放等多点操作。
- *麦克风*：接收、分析语音指令。
- *摄像头*：捕获图像，实现人脸识别、动作识别等高级交互。

3. 事件驱动的交互技术框架
- **事件驱动机制**：  
  程序不停地“监听”外部事件（如鼠标移动、键盘按键），当检测到事件发生时，自动调用相应的代码进行处理。
- **构成要素**：  
  - *事件源*：交互设备或输入方式（如鼠标、键盘）。
  - *事件对象*：记录事件的相关信息（如是哪一个鼠标按钮被按下、哪个按键被敲击）。
  - *事件响应/回调函数*：预先写好的程序段，只有对应事件发生时才被自动执行。
- **常见流程**：  
  “某个事件发生→系统检测并生成事件对象→触发指定响应函数”，这样实现程序的动态反应。
  
4. Processing中的交互基础
- **`setup()` 函数**：  
只在程序开始时运行一次，主要用于窗口尺寸、全局变量、资源加载等初始化设置。
- **`draw()` 函数**：  
程序主循环，每秒多次自动运行，使得动画、检测实时变化得以实现。
- **鼠标交互相关属性**：  
  - `mouse_x`, `mouse_y` 当前鼠标指针的坐标。
  - `mouse_pressed` 鼠标是否被按下，`mouse_button` 判断按下的是哪个按钮。
- **键盘交互相关属性**：  
  - `key` 当前按下的字符键，`key_code` 用于检测特殊按键（如上下左右等非字符键）。
  - `key_pressed` 检测当前是否有按键被按下。
- **基本实现方法**：  
在 `draw()` 中实时检测上述变量，实现“按下键盘/鼠标时，界面即时变化”，或通过事件函数（如 `mouse_pressed()`、`key_pressed()`）实现响应操作。



##### 课后练习


1. **如果你是一个寿司店店主，仿照事件-响应模式，设计一个接待流程。**  
   用“顾客进门—打招呼—点餐—制作—上餐—结账—送客”等作为事件触发点，分别描述每一步的响应和后续动作。
2. **如果你是咖啡馆服务员，仿照事件-响应模式，设计一个点单和制作流程。**
3. **如果你在银行办理业务，试用事件-响应模式描述你与大堂经理及窗口的互动过程。**
4. **假设你开发一个“智能提醒”App，参照 setup/draw 结构，尝试设计其初始化与持续运行的框架。**
5. **如果你构建一个“天气动态可视化”系统，如何用 setup/draw 的思路拆分代码结构和交互响应？**
6. **设计这个程序，不用代码实现**：  
   - 用 Processing 或 Py5 实现一个“涂鸦板”，支持鼠标绘画和按键变色。  
   - 实现一个按下不同键盘按键，背景颜色随之变化的小应用。  
   - 使用触摸板或者鼠标点击，让“笑脸表情”在画布上跟随手指移动。  
   - 记录并分析你调试时遇到的报错或bug，并给出解决过程。  
   - 设计并实现小型“打地鼠”或者“键盘打字训练”微型游戏，体会事件响应式交互的乐趣。**



##### 扩展知识


事件驱动系统（Event-Driven System），是计算机科学与软件工程设计中的核心架构模式之一。这种系统的运行流程完全由外部事件的发生来驱动，而非传统的线性执行流程。在现代软件架构设计中，有许多经典的设计模式与架构模式，常见的还有MVC（模型-视图-控制器）、MVVM（模型-视图-视图模型）、MVP（模型-视图-表示器）、观察者模式、发布-订阅模式等，分别用于不同的应用场景：MVC主要用于Web应用和桌面应用的分层架构，MVVM特别适合数据绑定密集的现代前端框架，而观察者模式和发布-订阅模式则是事件驱动系统的核心实现机制。这些设计模式都是交互设计实现的重要软件基础，为构建响应式、可维护的用户界面提供了理论支撑。  

可以阅读以下文档，获取最新的架构设计理念、实践案例、性能优化策略和行业最佳实践：  

- [AWS事件驱动架构文档](https://aws.amazon.com/cn/what-is/eda/)
- [Microsoft Azure事件驱动架构指南](https://learn.microsoft.com/zh-cn/azure/architecture/guide/architecture-styles/event-driven)
- [阿里云事件驱动架构文档](https://help.aliyun.com/zh/eventbridge/product-overview/eda)

在实际工程和软件开发领域，事件驱动系统广泛应用于操作系统、图形用户界面（GUI）、网络通信、游戏引擎、以及嵌入式系统等诸多场景。例如，操作系统需要对键盘输入、鼠标移动、外设插拔等硬件事件做出实时响应；Web 浏览器通过事件模型管理页面中的点击、滚动、按键等用户动作；[Node.js](https://nodejs.org/zh-cn/docs) 等平台则利用事件机制高效处理网络 I/O 和多用户并发。对于 Web 前端开发，[JavaScript](https://developer.mozilla.org/zh-CN/docs/Web/JavaScript/) 的事件机制（如 addEventListener、事件冒泡与捕获等）是学习交互编程的基础。这些不同场景下的事件驱动模型分别针对异步操作、实时反馈或复杂交互提供了高效、可扩展的技术基础。  

一个完整的交互系统不仅依靠先进的软件事件驱动框架，同时也离不开多样化的交互硬件。传统的计算机交互硬件主要包括鼠标和键盘，这些设备我们在前文编程实践中已经频繁使用。随着交互技术的不断发展，越来越多创新型设备被引入现代交互场景，例如蓝牙手柄、[Kinect](https://learn.microsoft.com/en-us/) 体感设备、[Leap Motion](https://github.com/leapmotion) 手势识别、虚拟现实（VR）控制器、眼动追踪仪等。这些现代交互硬件极大地丰富了输入方式和使用体验，使得交互设计能够从二维屏幕拓展到三维空间乃至多模态感知。在许多高端游戏、仿真、艺术装置中，对这些硬件的支持和驱动已经成为交互体验创新的关键。

在软件层面，图形界面（GUI）交互作为最常见的人机交互形式，近年来不断融合创新的设计理念。如抬头显示（HUD, Head-Up Display）、手势操作、语音交互、增强现实（AR）界面等新兴方式，正推动交互界面向更自然、更直观、更高效的方向发展。例如，汽车工业中的 HUD 可以在挡风玻璃上实时投射关键信息，用户无需低头即可掌握数据；智能手机和平板电脑广泛使用基于多点触控和语音助手的交互方式，为用户带来无缝的操作体验。这些设计的共同目标，是让用户对数字系统的操控变得更加便捷灵活，最大程度减少学习成本和物理负担。

此外，除 Processing 以外，当前常用的创意编程平台还有许多，均为交互艺术、多媒体开发和创客教育领域的重要工具。例如：  
- [openFrameworks](https://openframeworks.cc/)：基于 C++ 的开源创意编程工具箱，广泛应用于新媒体艺术、音视频处理和实时互动项目。  
- [TouchDesigner](https://derivative.ca/)：面向可视化编程和实时投影互动设计的强大工具，经常用于现场视觉、灯光控制、大型艺术展演等。  
- [p5.js](https://p5js.org/)：Processing 的 JavaScript 版本，非常适合网页上的交互艺术与教育用途。  
- [vvvv](https://vvvv.org/)：节点式编程环境，适合实时多媒体和数据可视化开发。  
- [Max/MSP/Jitter](https://cycling74.com/)：专注音乐、声音与多媒体互动，尤其适合电子音乐和装置艺术。  
- [Unity3D](https://unity.com/)：虽然以游戏开发著称，但其丰富的可视化、物理仿真和多硬件接口，也常用于交互艺术和实验项目。  

这些平台结合了多语言支持、丰富的硬件接口及社区资源，为交互设计师、艺术家与程序员提供了广阔的实践与创新空间。阅读各平台的官网、社区案例和开源项目，有助于扩展视野、汲取灵感、掌握更前沿的交互技术。



##### 练习题提示


4. 用 setup 初始化（如加载提醒列表），draw 持续更新界面信息，检测鼠标或按键交互。  
5. 用 setup 初始化（如加载天气数据），draw 持续更新界面信息，检测鼠标或按键交互。  
6. 系统设计提示：  
- “涂鸦板”用 draw 监听 mouse_pressed 绘制曲线，key_pressed 改变颜色。
- 背景变色用 key 事件更新背景色变量。
- “表情跟随”可用 mouse_x, mouse_y 实时设置表情坐标。
- 报错常见如拼写错误、未定义变量、事件函数名写错，建议对照报错行数排查。
- 微型游戏用 mouse_pressed 检测击打、打字检测结合 key_pressed 实现逻辑。所有实践均需灵活运用 setup/draw 及事件检测。